In [7]:
import argparse
import os
import shutil
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

# Configure Model

In [8]:
model="llama3:latest"
ollama_url="http://localhost:11434"#"http://192.168.2.2:11434"
embeddings = OllamaEmbeddings(model=model,base_url=ollama_url)
ollama = Ollama(model=model,base_url=ollama_url)
db = Chroma(persist_directory="./chroma", embedding_function=embeddings)

# Helpers

In [9]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

def add_to_chroma(chunks: list[Document], db):
    chunks_with_ids = calculate_chunk_ids(chunks)
    existing_items = db.get(include=[])
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
    else:
        print("✅ No new documents to add")


def calculate_chunk_ids(chunks):
    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index
    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

def query(query_text: str, model, db):
    PROMPT_TEMPLATE = """
Beantworte die folgende Frage nur anhand des gegebenen Kontext:

{context}

---

Beantworte folgende Frage anhand des obigen Kontext: {question}
"""
    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    response_text = ollama.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    print(context_text)
    return response_text

# Load the PDF into the Vector Store. 
Takes reeeeeally long. Must only be done once

In [11]:
document_loader = PyPDFDirectoryLoader('./data')
docs = document_loader.load()
chunks = split_documents(docs)
add_to_chroma(chunks, db)

Number of existing documents in DB: 8957
✅ No new documents to add


# Query the Model and Database

In [10]:
query("Welche Maßnahmen sind sinnvoll bei einer steigenden Infektionsrate, einer hohen Impfrate und einem geringen Sterbegeschehen?",ollama,db)

Response: Eine steigende Infektionsrate, eine hohe Impfrate und ein geringes Sterbegeschehen sprechen für eine Situation, in der sich die Gesellschaft langsam aber sicher wieder normalisiert. In diesem Kontext sind folgende Maßnahmen sinnvoll:

1. Fortsetzung der Impfungen: Da die Impfrate hoch ist und die Nebenwirkungen der Impfstoffe bekannt sind, sollte die Impfung weiterhin als wirksames Mittel gegen die Verbreitung des Virus genutzt werden.
2. Überwachung von Risikofaktoren: Es könnte sinnvoll sein, eine CWA-basierte Umfrage bei bestimmten Personen zu Risikofaktoren durchzuführen, um die Verbreitung des Virus besser verstehen und geeignete Maßnahmen ergreifen zu können.
3. Erneute Öffnung von Schulen und Kitas: Da Studien zeigen, dass Kinder nicht mehr wie erwachsene Personen ansteckbar sind und auch keine anderen Personen infiziert haben, wenn sie selbst infiziert waren, könnte die Wiedereröffnung von Schulen und Kitas sinnvoll sein.
4. Toolbox für Länder erstellen: Es wäre ratsa

'Eine steigende Infektionsrate, eine hohe Impfrate und ein geringes Sterbegeschehen sprechen für eine Situation, in der sich die Gesellschaft langsam aber sicher wieder normalisiert. In diesem Kontext sind folgende Maßnahmen sinnvoll:\n\n1. Fortsetzung der Impfungen: Da die Impfrate hoch ist und die Nebenwirkungen der Impfstoffe bekannt sind, sollte die Impfung weiterhin als wirksames Mittel gegen die Verbreitung des Virus genutzt werden.\n2. Überwachung von Risikofaktoren: Es könnte sinnvoll sein, eine CWA-basierte Umfrage bei bestimmten Personen zu Risikofaktoren durchzuführen, um die Verbreitung des Virus besser verstehen und geeignete Maßnahmen ergreifen zu können.\n3. Erneute Öffnung von Schulen und Kitas: Da Studien zeigen, dass Kinder nicht mehr wie erwachsene Personen ansteckbar sind und auch keine anderen Personen infiziert haben, wenn sie selbst infiziert waren, könnte die Wiedereröffnung von Schulen und Kitas sinnvoll sein.\n4. Toolbox für Länder erstellen: Es wäre ratsam, e